In [48]:
# Scripts looks for WordPress websites in Google Search
# Creation date: Sept 2022
# Last modification: 05 Jan 2023

# IDEA: Save non-WordPress sites - maybe I can offer them something too
# IDEA: Drop .gov and .edu domains - done in "drop_gov_and_edu.ipynb" (for emails, but should also be done for domains)
# IDEA: Maybe the script should append currently exisitng excel so it can be stopped any time


from bs4 import BeautifulSoup
import openpyxl
import requests, re
import pandas as pd
import numpy as np
from urllib.parse import urlsplit
from collections import deque
from datetime import datetime, date
from time import sleep

search_for = "car rental" #Change only this string
search_for = search_for.replace(" ", "+")
headers = {
    "User-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"
}
cities = ['Clovis']#, 'Seattle', 'Los Angeles', 'Tampa', 'Brooklyn', 'Phoenix','Evansville','Springfield','Independence','Ann Arbor','Hartford','Hillsboro','Abilene']
# cities = ['Roanoke','Nampa ','Edinburg ','Clinton ','Federal Way ','New Bedford ','Lees Summit ','Lynn ','Chico ','Quincy ','Davenport ','Wichita Falls ',
# 'Vacaville ','Menifee ','Norwalk ','Spokane Valley ','South Bend ','Woodbridge ','Rialto ','Rio Rancho ','Allen ','Daly City ','Jurupa Valley','Concord ',
# 'Brockton ','San Mateo ','Davie','Tyler ','El Cajon ','Hillsboro ','Renton ','Burbank ','Green Bay ','South Fulton ','Edison ','Inglewood ','Sandy Springs ',
# 'Boulder ','Centennial ','Sparks ','Greeley ','El Monte ','West Covina ','Santa Maria ','Dearborn ','Temecula ','Everett ','Ventura','Murrieta ','Sugar Land ',
# 'Las Cruces ','Miami Gardens ','Lewisville ','Pueblo ','Costa Mesa ','Pompano Beach ','Lakeland ','Lansing ','Peoria ','Broken Arrow ','High Point ','Gresham ',
# 'Downey ','League City ','Springfield ','Waterbury ','Odessa ','Carlsbad ','Elgin ','North Charleston ','Provo ','Beaumont ','Antioch ','Wilmington ','Lowell ',
# 'Manchester ','Westminster ','Richmond ','West Jordan ','Billings ','Clearwater ','Evansville ','West Palm Beach ','Meridian ','Cambridge ','Round Rock ','Richardson ',
# 'Palm Bay ','Fairfield ','Clovis ','College Station ','Hartford ','Lafayette','Rochester ','Independence ','Ann Arbor ','Berkeley ','Arvada ','Abilene ']

def try_except(func, *arg):
    try:
        result = func(*arg)
        print(func.__name__, ":", *arg, "=", result)
        return result
    except Exception as e:
        print("1:", e)
        return "N/a"

def check_if_wordpress(url):
    r = requests.get(url, headers=headers, timeout=5)
    if "wordpress" in r.text:
        return True
    else:
        return False

# def scrap_href(link):
#     links = link.a["href"]
#     links = links.split("/", 3)[:3]
#     links[1] = "//"
#     links[0:3] = [''.join(links[0:3])]
#     return links[0]
       
#Scrapping URLs
is_wordpress = []
links_list = []
skips_number = 0
for city in cities:
    n = 1
    exeptions_number = 0
    url = f"https://www.google.com/search?q={search_for}+{city}"
    while n < 10: #number of pages to scrap should be 10
        progress = ((cities.index(city) * 10 + n) / (len(cities) * 10)) * 100 # Counting progress %
        print(city, n, "/ 10 | Progress: ", "{:.2f}".format(progress), "%")
        try:
          if n == 1:
              response = requests.get(url, headers=headers, timeout=5)
              soup = BeautifulSoup(response.text, 'lxml')
          else: # It will be executed when n >= 2
              next_page_url = soup.find("a", {"aria-label":f"Page {n}"})["href"]
              response = requests.get(f"https://google.com{next_page_url}", headers=headers)
              soup = BeautifulSoup(response.text, 'lxml')
          for link in soup.find_all("div", {"class":"g"}): # There should be 9-10 divs (results) with this class
              links = link.a["href"] # Taking only "a.href" (links) from div
              links = links.split("/", 3)[:3] # Taking only link till 3rd "/" (https://smtng.com/...)
              links[1] = "//"
              links[0:3] = [''.join(links[0:3])]
              links_list.append(links[0]) # Saving link to list
              is_wordpress.append(try_except(check_if_wordpress, links[0]))
        except Exception as e:
            print("2:", e) #If "NoneType" error - probably google blocked the IP
            exeptions_number += 1
            if exeptions_number > 2: #If 3 errors in one "city" scrap then break 'while' loop
                sleep(10)
                if n == 1: #If error occures in 1st loop execution (n) then add skips_number
                    skips_number += 1
                print(f"Exeptions_number is: {exeptions_number}. Breaking 'while' loop.")
                break
            else:
                sleep(2)
            pass
        sleep(0.5)
        n += 1
    if skips_number > 2: #If 3 errors in 1st loop execution (n) then break 'for' loop
        print(f"skips_number is: {skips_number}. Breaking 'for' loop.")
        break


dict = {
    "Website" : links_list,
    "WordPress" : is_wordpress,
    #"Keyword" : search_for,
}

df = pd.DataFrame(dict)
df = df[~df['Website'].isin(['https://allegro.pl', 'https://www.olx.pl', 'https://pl.wikipedia.org', 'https://www.facebook.com', 'https://maps.google.com',
'https://www.yelp.com', 'https://www.indeed.com'])]
df = df.drop_duplicates(subset=['Website'])
df_wp = df[df['WordPress'].isin([True])]
df_non_wp = df[df['WordPress'].isin([False])]
df_wp
now = datetime.now().strftime("%d-%m-%Y_%H-%M")
# df_wp.to_excel(rf"C:\Projekty\Coding\wp_security_report\leads\output\g_scrap_wp_{now}.xlsx", index=False)
# df_non_wp.to_excel(rf"C:\Projekty\Coding\wp_security_report\leads\output_non_wp\g_scrap_non_wp_{now}.xlsx", index=False)


Clovis 1 / 10 | Progress:  10.00 %
Clovis 2 / 10 | Progress:  20.00 %
2: 'NoneType' object is not subscriptable
Clovis 3 / 10 | Progress:  30.00 %
2: 'NoneType' object is not subscriptable
Clovis 4 / 10 | Progress:  40.00 %
2: 'NoneType' object is not subscriptable
Clovis 5 / 10 | Progress:  50.00 %
2: 'NoneType' object is not subscriptable
Clovis 6 / 10 | Progress:  60.00 %
2: 'NoneType' object is not subscriptable
Clovis 7 / 10 | Progress:  70.00 %
2: 'NoneType' object is not subscriptable
Clovis 8 / 10 | Progress:  80.00 %
2: 'NoneType' object is not subscriptable
Clovis 9 / 10 | Progress:  90.00 %
2: 'NoneType' object is not subscriptable
